# SQL

In [22]:
import sqlite3
import os
import pandas as pd
from google.colab import drive # <- Importar para Drive


# --- PASO 1: MONTAR GOOGLE DRIVE ---
print("Montando Google Drive...")
drive.mount('/content/drive')
print("Google Drive montado.")

# --- PASO 2: CARGAR LA RUTA EXISTENTE ---
RUTA_CARPETA_DATOS = '/content/drive/MyDrive/Universidad Siglo 21/Seminiario_Final_CienciaDeDatos/TP4/Base de datos'
NOMBRE_BASE_DATOS = "CFO_SAP_PYL.db"
RUTA_BASE_DATOS = os.path.join(RUTA_CARPETA_DATOS, NOMBRE_BASE_DATOS)

# --- PASO 3: RECONECTAR A LA BASE DE DATOS ---
try:
    conn = sqlite3.connect(RUTA_BASE_DATOS)
    cursor = conn.cursor()
    print(f"\n¡Conexión exitosa a la base de datos existente en:\n{RUTA_BASE_DATOS}")

except Exception as e:
    print(f"\nOcurrió un error al conectar la base de datos: {e}")

Montando Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Google Drive montado.

¡Conexión exitosa a la base de datos existente en:
/content/drive/MyDrive/Universidad Siglo 21/Seminiario_Final_CienciaDeDatos/TP4/Base de datos/CFO_SAP_PYL.db


In [23]:
print("\n--- CONTENIDO DE LA TABLA ---")
conn = sqlite3.connect(RUTA_BASE_DATOS)
cursor = conn.cursor()
cursor.execute("SELECT * FROM CFO_SAP_PYL limit 10;")

registros = cursor.fetchall()

if not registros:
    print("La tabla está vacía.")
else:
    for fila in registros:
        print(fila)


--- CONTENIDO DE LA TABLA ---
(1, 33362.98, 'COGS', 'LOCAL EBIT', 'BRASIL', 'REALIZADO', 'DEFLATED', 2023, 'Q3', 202308, 'ONLINE PAYMENTS', 'FINTECH SERVICES', None, None)
(2, 31818.16, 'TAX', 'LOCAL EBIT', 'CHILE', 'REALIZADO', 'LOCAL', 2023, 'Q4', 202308, 'ONLINE PAYMENTS', None, 'REGIONAL', None)
(3, 97133.75, 'EXPENSES', 'LOCAL EBIT', 'ARGENTINA', 'PLAN', 'LOCAL', 2023, 'Q4', 202308, 'CREDITS', 'FINTECH SERVICES', 'CORPORATE', 'BRASIL')
(4, 25715.32, 'COGS', 'LOCAL EBIT', 'BRASIL', 'PLAN', 'LOCAL', 2023, 'Q4', 202304, 'ONLINE PAYMENTS', 'E-COMMERCE', None, 'ARGENTINA')
(5, 6446.37, 'COGS', 'LOCAL EBIT', 'BRASIL', 'PLAN', 'USD CC', 2023, 'Q4', 202303, None, 'E-COMMERCE', 'CORPORATE', None)
(6, 29921.52, 'NET REVENUES', 'LOCAL EBIT', 'BRASIL', 'REALIZADO', 'USD', 2023, 'Q3', 202308, 'CREDITS', 'E-COMMERCE', 'CORPORATE', 'LATAM')
(7, 64762.06, 'TAX', 'LOCAL EBIT', 'CHILE', 'REALIZADO', 'USD', 2023, 'Q2', 202310, 'RETAIL', 'E-COMMERCE', 'CORPORATE', None)
(8, 82104.22, 'NET REVENUES',

In [3]:
# Ejecutar query y traer resultados a un DataFrame
query = """
SELECT PYL0, ANIO, MES_ID, ESCENARIO, MONEDA, ALLOCATION, DICCIONARIO_COUNTRY ,
  SUM(CASE WHEN IMPORTE IS NULL THEN 0 ELSE IMPORTE END) AS IMPORTE_SOL
FROM
  CFO_SAP_PYL
WHERE
  PYL0 = 'NET REVENUES'
  AND ALLOCATION ='REGIONAL'
  AND ESCENARIO = 'REALIZADO'
  AND ANIO = 2024
  AND DICCIONARIO_COUNTRY = 'ARGENTINA'
GROUP BY
  PYL0, ANIO, MES_ID, ESCENARIO, MONEDA, ALLOCATION, DICCIONARIO_COUNTRY

"""
df = pd.read_sql_query(query, conn)

# Mostrar resultados
df.head(20)


,PYL0,ANIO,MES_ID,ESCENARIO,MONEDA,ALLOCATION,DICCIONARIO_COUNTRY,IMPORTE_SOL
0,NET REVENUES,2024,202402,REALIZADO,LOCAL,REGIONAL,ARGENTINA,816325.07
1,NET REVENUES,2024,202404,REALIZADO,DEFLATED,REGIONAL,ARGENTINA,195789.76
2,NET REVENUES,2024,202404,REALIZADO,USD,REGIONAL,ARGENTINA,850552.91
3,NET REVENUES,2024,202405,REALIZADO,LOCAL,REGIONAL,ARGENTINA,48860.91
4,NET REVENUES,2024,202405,REALIZADO,USD CC,REGIONAL,ARGENTINA,798424.93
5,NET REVENUES,2024,202406,REALIZADO,DEFLATED,REGIONAL,ARGENTINA,16648.16
6,NET REVENUES,2024,202407,REALIZADO,LOCAL,REGIONAL,ARGENTINA,202387.99
7,NET REVENUES,2024,202407,REALIZADO,USD,REGIONAL,ARGENTINA,891665.17
8,NET REVENUES,2024,202408,REALIZADO,DEFLATED,REGIONAL,ARGENTINA,402824.86
9,NET REVENUES,2024,202409,REALIZADO,DEFLATED,REGIONAL,ARGENTINA,430792.59


#LLM

##instalación inicial

In [ ]:
!pip install streamlit groq
!apt-get install cloudflared

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cloudflared


##versión1

In [ ]:
%%writefile app.py
import streamlit as st
from groq import Groq
from typing import Generator

st.set_page_config(page_title="Smart Finace", page_icon="🤖")
st.title("🤖 Smart Finace")

# 🔑 Pega tu API Key de Groq aquí
client = Groq(api_key="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")


# Lista de modelos
modelos = ['llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'mixtral-8x7b-32768']

#parModelo = st.sidebar.selectbox('Modelos', options=modelos, index=0)

def generate_chat_responses(chat_completion) -> Generator[str, None, None]:
    for chunk in chat_completion:
        if chunk.choices[0].delta.content:
            yield chunk.choices[0].delta.content

# Inicializamos historial

if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "Eres un asistente que siempre responde en español, sin importar el idioma de la pregunta."}
    ]

# Mostrar historial en pantalla
with st.container():
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

# Selección de modelo
parModelo = st.sidebar.selectbox('Modelos disponibles', options=modelos, index=0)

# Campo de input estilo chat
prompt = st.chat_input("Escribe tu pregunta...")

if prompt:
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    try:
        chat_completion = client.chat.completions.create(
            model=parModelo,
            messages=[
                {"role": m["role"], "content": m["content"]}
                for m in st.session_state.messages
            ],
            stream=True
        )

        with st.chat_message("assistant"):
            chat_responses_generator = generate_chat_responses(chat_completion)
            full_response = st.write_stream(chat_responses_generator)

        st.session_state.messages.append({"role": "assistant", "content": full_response})

    except Exception as e:
        st.error(e)


Writing app.py


##versión2

In [ ]:
%%writefile app.py
import streamlit as st
from groq import Groq
from typing import Generator
import sqlite3
import pandas as pd

st.set_page_config(page_title="Smart Finance", page_icon="🤖")
st.title("🤖 Smart Finance")

# 🔑 Pega tu API Key de Groq aquí
client = Groq(api_key="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")

# Ruta a la base SQLite
DB_PATH = "/mnt/data/CFO_SAP_PYL.db"

# Lista de modelos
modelos = ['llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'mixtral-8x7b-32768']

def generate_chat_responses(chat_completion) -> Generator[str, None, None]:
    for chunk in chat_completion:
        if chunk.choices[0].delta.content:
            yield chunk.choices[0].delta.content

# Inicializamos historial
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "Eres un asistente financiero. Siempre responde en español, con explicaciones simples para contadores. Si la pregunta es sobre datos, genera SQL para la tabla CFO_SAP_PYL, ejecuta la consulta y devuelve los resultados explicados en lenguaje natural."}
    ]

# Mostrar historial en pantalla
with st.container():
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

# Selección de modelo
parModelo = st.sidebar.selectbox('Modelos disponibles', options=modelos, index=0)

# Campo de input estilo chat
prompt = st.chat_input("Haz tu consulta financiera...")

if prompt:
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    try:
        # Paso 1: Generar SQL con el LLM
        sql_prompt = f"""
Eres un asistente financiero especializado en contabilidad y control de gestión. Tu objetivo es ayudar a usuarios no técnicos a obtener información de la base de datos CFO_SAP_PYL sin que necesiten saber SQL.

Sigue este flujo para cada pregunta que recibas:

1. Recibes la pregunta en lenguaje natural de un usuario (ej. “¿Cuál fue el PYL0 total por país y año?”).

2. Genera SOLO una query SQL **válida para SQLite** que responda la pregunta usando la tabla CFO_SAP_PYL, considerando los siguientes campos disponibles:
[IMPORTE, PYL0, DICCIONARIO_LOCAL_EBIT, DICCIONARIO_COUNTRY, ESCENARIO, MONEDA, ANIO, DICCIONARIO_Q, MES_ID, DICCIONARIO_SUBBU, DICCIONARIO_MACRO_BU, ALLOCATION, SUBSEGMENTO].

3. Reglas para la query:
   - No uses `SELECT *`; enumera solo las columnas necesarias.
   - Maneja valores NULL correctamente.
   - Limita la salida a **máximo 100 filas** para evitar consultas pesadas.
   - La query debe ser completamente ejecutable en SQLite.
   - No agregues comentarios ni texto, SOLO devuelve la SQL.

4. Una vez generada la SQL, ejecútala contra la base de datos.

5. Luego, toma el resultado de la query y tradúcelo a **lenguaje natural sencillo**, explicando claramente lo que muestran los datos:
   - Resume resultados si son muchas filas.
   - Usa términos comprensibles para contadores y personal de contraloría.
   - No uses jerga técnica de SQL ni términos de programación, sino mas bien legueje contable/financiero.

6. Devuelve al usuario únicamente el **resumen en lenguaje natural**, pero permite que, si se desea, también se vea la SQL ejecutada en un bloque aparte.

Ejemplo de flujo:

Usuario: “¿Cuál fue el PYL0 total por país y año para 2025?”
LLM:
- Genera SQL válida para SQLite:
```sql
SELECT DICCIONARIO_COUNTRY, ANIO, SUM(PYL0) AS Total_PYL0
FROM CFO_SAP_PYL
WHERE ANIO = 2025
GROUP BY DICCIONARIO_COUNTRY, ANIO
LIMIT 100;


        Pregunta: {prompt}
        """

        sql_response = client.chat.completions.create(
            model=parModelo,
            messages=[{"role": "user", "content": sql_prompt}],
            stream=False
        )

        sql_query = sql_response.choices[0].message.content.strip()

        # Mostrar SQL generado
        with st.expander("🔎 SQL generado por el asistente"):
            st.code(sql_query, language="sql")

        # Paso 2: Ejecutar SQL contra la base
        conn = sqlite3.connect(DB_PATH)
        try:
            df = pd.read_sql_query(sql_query, conn)
        except Exception as e:
            st.error(f"Error ejecutando la SQL: {e}")
            df = None
        conn.close()

        if df is not None and not df.empty:
            st.dataframe(df.head(50))  # mostramos tabla

            # Paso 3: Explicar el resultado en lenguaje natural
            explain_prompt = f"""
            Explica este resultado de forma simple y clara para contadores no técnicos.
            Pregunta original: {prompt}
            Resultado de la query (primeras filas): {df.head(5).to_dict()}
            """

            explanation = client.chat.completions.create(
                model=parModelo,
                messages=[{"role": "user", "content": explain_prompt}],
                stream=False
            )

            explanation_text = explanation.choices[0].message.content.strip()

            with st.chat_message("assistant"):
                st.markdown(explanation_text)

            st.session_state.messages.append({"role": "assistant", "content": explanation_text})
        else:
            st.warning("No se encontraron resultados para tu consulta.")

    except Exception as e:
        st.error(e)


Overwriting app.py


##version3

In [ ]:
%%writefile app.py
import streamlit as st
from groq import Groq
from typing import Generator
import sqlite3
import pandas as pd

st.set_page_config(page_title="Smart Finance", page_icon="🤖")
st.title("🤖 Smart Finance")

# 🔑 Pega tu API Key de Groq aquí
client = Groq(api_key="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")

# Ruta a la base SQLite
DB_PATH = "/mnt/data/CFO_SAP_PYL.db"

# Lista de modelos
modelos = ['llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'mixtral-8x7b-32768']

def generate_chat_responses(chat_completion) -> Generator[str, None, None]:
    for chunk in chat_completion:
        if chunk.choices[0].delta.content:
            yield chunk.choices[0].delta.content

# Inicializamos historial
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "Eres un asistente financiero. Siempre responde en español, con explicaciones simples para contadores. Si la pregunta es sobre datos, genera SQL para la tabla CFO_SAP_PYL, ejecuta la consulta y devuelve los resultados explicados en lenguaje natural."}
    ]

# Mostrar historial en pantalla
with st.container():
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

# Selección de modelo
parModelo = st.sidebar.selectbox('Modelos disponibles', options=modelos, index=0)

# Campo de input estilo chat
prompt = st.chat_input("Haz tu consulta financiera...")

if prompt:
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    try:
        # Paso 1: Generar SQL con el LLM

        sql_prompt = f"""
Eres un asistente financiero experto. Genera SOLO una SQL válida para SQLite.
Genera SOLO una SQL válida para SQLite y solo podés usar los siguientes campos de la tabla CFO_SAP_PYL:

[IMPORTE, PYL0, DICCIONARIO_LOCAL_EBIT, DICCIONARIO_COUNTRY, ESCENARIO, MONEDA, ANIO, DICCIONARIO_Q, MES_ID, DICCIONARIO_SUBBU, DICCIONARIO_MACRO_BU, ALLOCATION, SUBSEGMENTO]

Reglas:
- Tu respuesta debe contener **solo la SQL**. Nada más.
- No inventes campos.
- No uses SELECT *.
- Si usás SUM/AVG, no pongas LIMIT.
- Maneja NULL con CASE WHEN necesario.
- Limitá resultados solo cuando se pidan listados, no en agregados.

Pregunta: {prompt}
"""

        sql_response = client.chat.completions.create(
            model=parModelo,
            messages=[{"role": "user", "content": sql_prompt}],
            stream=False
        )

        sql_query = sql_response.choices[0].message.content.strip()

        # 🧹 Limpieza de formato Markdown del modelo
        sql_query = sql_query.replace("```sql", "").replace("```", "").strip()


        # Mostrar SQL generado
        with st.expander("🔎 SQL generado por el asistente"):
            st.code(sql_query, language="sql")

        # Paso 2: Ejecutar SQL contra la base
        conn = sqlite3.connect(DB_PATH)
        try:
            df = pd.read_sql_query(sql_query, conn)
        except Exception as e:
            st.error(f"Error ejecutando la SQL: {e}")
            df = None
        conn.close()

        if df is not None and not df.empty:
            st.dataframe(df.head(50))  # mostramos tabla

            # Paso 3: Explicar el resultado real en lenguaje natural
            explain_prompt = f"""
Eres un asistente financiero que explica resultados de manera simple y clara para contadores y contraloría.
Pregunta original: {prompt}
Resultado real de la query (primeras filas): {df.head(5).to_dict()}
Explica solo en lenguaje natural, sin SQL ni jerga técnica.
"""

            explanation = client.chat.completions.create(
                model=parModelo,
                messages=[{"role": "user", "content": explain_prompt}],
                stream=False
            )

            explanation_text = explanation.choices[0].message.content.strip()

            with st.chat_message("assistant"):
                st.markdown(explanation_text)

            st.session_state.messages.append({"role": "assistant", "content": explanation_text})
        else:
            st.warning("No se encontraron resultados para tu consulta.")

    except Exception as e:
        st.error(e)


Overwriting app.py


##version4

In [17]:
!pip install streamlit groq
!apt-get install cloudflared


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
E: Unable to locate package cloudflared


In [24]:
%%writefile app.py
import streamlit as st
from groq import Groq
from typing import Generator
import sqlite3
import pandas as pd

st.set_page_config(page_title="Smart Finance", page_icon="🤖")
st.title("🤖 Smart Finance")

# 🔑 Pega tu API Key de Groq aquí
client = Groq(api_key="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")

# Ruta a la base SQLite
DB_PATH = NOMBRE_BASE_DATOS

# Lista de modelos
modelos = ['llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'mixtral-8x7b-32768']

def generate_chat_responses(chat_completion) -> Generator[str, None, None]:
    for chunk in chat_completion:
        if chunk.choices[0].delta.content:
            yield chunk.choices[0].delta.content

# Inicializamos historial
if "messages" not in st.session_state:
    st.session_state.messages = [
        {"role": "system", "content": "Eres un asistente financiero. Siempre responde en español, con explicaciones simples para contadores. Si la pregunta es sobre datos, genera SQL para la tabla CFO_SAP_PYL, ejecuta la consulta y devuelve los resultados explicados en lenguaje natural."}
    ]

# Mostrar historial en pantalla
with st.container():
    for message in st.session_state.messages:
        with st.chat_message(message["role"]):
            st.markdown(message["content"])

# Selección de modelo
parModelo = st.sidebar.selectbox('Modelos disponibles', options=modelos, index=0)

# Campo de input estilo chat
prompt = st.chat_input("Haz tu consulta financiera...")

if prompt:
    st.chat_message("user").markdown(prompt)
    st.session_state.messages.append({"role": "user", "content": prompt})

    try:
        # Paso 1: Generar SQL con el LLM

        sql_prompt = f"""
Eres un asistente financiero experto en generar consultas SQL para analistas contables.
Tu tarea es generar SOLO una consulta SQL válida para SQLite sobre la tabla CFO_SAP_PYL, sin ninguna explicación adicional.

Estructura de la tabla CFO_SAP_PYL:

Campos disponibles (usa solo estos):
[IMPORTE, PYL0, DICCIONARIO_LOCAL_EBIT, DICCIONARIO_COUNTRY, ESCENARIO, MONEDA, ANIO, DICCIONARIO_Q, MES_ID, DICCIONARIO_SUBBU, DICCIONARIO_MACRO_BU, ALLOCATION, SUBSEGMENTO]

Valores válidos para algunos campos:
- ESCENARIO: 'REALIZADO', 'PLAN'
- MONEDA: 'USD', 'LOCAL', 'DEFLATED', 'USD CC'
- DICCIONARIO_Q: 'Q1', 'Q2', 'Q3', 'Q4'
- PYL0: 'NET REVENUES', 'COGS', 'EXPENSES', 'TAX'
- DICCIONARIO_COUNTRY y SUBSEGMENTO: 'ARGENTINA', 'BRASIL', 'CHILE', 'COLOMBIA', 'URUGUAY', 'VENEZUELA'
- DICCIONARIO_SUBBU: 'RETAIL', 'CREDITS', 'ONLINE PAYMENTS', 'POINT'
- DICCIONARIO_MACRO_BU: 'FINTECH SERVICES', 'E-COMMERCE'
- ALLOCATION: 'CORPORATE', 'REGIONAL'

Reglas obligatorias:
- Genera SOLO el código SQL. No incluyas explicaciones, encabezados ni markdown.
- No inventes campos ni valores que no existen.
- No uses SELECT *.
- Usá CASE WHEN para manejar posibles valores NULL si hay cálculos.
- Si la consulta requiere agregaciones como SUM, no uses LIMIT.
- Solo aplica filtros(where) explícitos para año, trimestre, mes, moneda, escenario o país si están presentes en la pregunta.
- Si la pregunta dice "YTD a [mes]", usá `MES_ID <= AAAAMM` dentro del mismo ANIO.
- Si se pide comparar años o trimestres, hacelo usando múltiples filtros con OR o agrupaciones por ANIO o DICCIONARIO_Q.
- Si la pregunta requiere desglose por país o subbu, usá GROUP BY apropiado.

Pregunta del usuario:{prompt}

"""

        sql_response = client.chat.completions.create(
            model=parModelo,
            messages=[{"role": "user", "content": sql_prompt}],
            stream=False
        )

        sql_query = sql_response.choices[0].message.content.strip()

        # 🧹 Limpieza de formato Markdown del modelo
        sql_query = sql_query.replace("```sql", "").replace("```", "").strip()


        # Mostrar SQL generado
        with st.expander("🔎 SQL generado por el asistente"):
            st.code(sql_query, language="sql")

        # Paso 2: Ejecutar SQL contra la base
        conn = sqlite3.connect(DB_PATH)
        try:
            df = pd.read_sql_query(sql_query, conn)
        except Exception as e:
            st.error(f"Error ejecutando la SQL: {e}")
            df = None
        conn.close()

        if df is not None and not df.empty:
            st.dataframe(df.head(50))  # mostramos tabla

            # Paso 3: Explicar el resultado real en lenguaje natural
            explain_prompt = f"""
Eres un asistente financiero que explica resultados de manera simple y clara para contadores y contraloría.
Pregunta original: {prompt}
Resultado real de la query (primeras filas): {df.head(5).to_dict()}
Explica solo en lenguaje natural, sin SQL ni jerga técnica.
"""

            explanation = client.chat.completions.create(
                model=parModelo,
                messages=[{"role": "user", "content": explain_prompt}],
                stream=False
            )

            explanation_text = explanation.choices[0].message.content.strip()

            with st.chat_message("assistant"):
                st.markdown(explanation_text)

            st.session_state.messages.append({"role": "assistant", "content": explanation_text})
        else:
            st.warning("No se encontraron resultados para tu consulta.")

    except Exception as e:
        st.error(e)

Overwriting app.py


# Servidor

In [25]:
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
!chmod +x cloudflared

In [26]:
!streamlit run app.py &>/dev/null &

In [ ]:
!./cloudflared tunnel --url http://localhost:8501 --no-autoupdate

2025-11-19T07:47:35Z INF Thank you for trying Cloudflare Tunnel. Doing so, without a Cloudflare account, is a quick way to experiment and try it out. However, be aware that these account-less Tunnels have no uptime guarantee, are subject to the Cloudflare Online Services Terms of Use (https://www.cloudflare.com/website-terms/), and Cloudflare reserves the right to investigate your use of Tunnels for violations of such terms. If you intend to use Tunnels in production you should use a pre-created named tunnel by following: https://developers.cloudflare.com/cloudflare-one/connections/connect-apps
2025-11-19T07:47:35Z INF Requesting new quick Tunnel on trycloudflare.com...
2025-11-19T07:47:37Z INF +--------------------------------------------------------------------------------------------+
2025-11-19T07:47:37Z INF |  Your quick Tunnel has been created! Visit it at (it may take some time to be reachable):  |
2025-11-19T07:47:37Z INF |  https://enhanced-cause-mia-architects.trycloudflare.c